In [12]:
from subprocess import call 
from glob import glob 
import pandas 

well_map = 'kunkelWellCSV_r18bf4kyjjtv8.csv'

# merge and align the sequences 
call(['bash', 'merge_and_align.bash', well_map ])

# diff and decide 
df = pandas.read_csv( well_map )
df.set_index( 'mutant name', inplace=True )

picks = [] 
for fil in glob( 'out/*diff' ):
  with open( fil ) as fn:
    sub, que = fn.readlines()[0].split( '\t' )   
    d = '+'.join( [ '{}{}{}'.format(i, n+1, j).lower() for (n, (i, j)) in enumerate( zip( sub, que ) ) if i != j ] ) 
    mutant_label = fil.split( '/' )[1].split( '.' )[0]
    mutant_name = mutant_label.split( '-' )[0]
    if mutant_name == d:
      picks.append( ( mutant_label, df.loc[ mutant_label ].well ) )

clones = pandas.DataFrame( picks, columns=['clone', 'well'] )
print 'Found that {} out of {} clones are correct ({}%)'.format( len( clones ), len( df ), 100 * len( clones ) / len( df ) )
clones.to_csv( 'clones.csv' )

Found that 26 out of 60 clones are correct (43%)
